# Análise de sentimento utilizando NLTK e modelo pré-treinado

In [ ]:
!pip install nltk

# Importação de bibliotecas que podem ser necessárias

In [ ]:
from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [ ]:
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [59]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\lopx1\AppData\Roaming\nltk_data...


True

# Analisando as bases

In [ ]:
df = pd.read_csv('NoThemeTweets.csv', encoding='utf-8')

In [64]:
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [91]:
df.sentiment.value_counts()

Negativo    517125
Positivo    259628
Name: sentiment, dtype: int64

In [ ]:
df.drop_duplicates(['tweet_text'], inplace=True)

In [70]:
tweets_positive = df[df['sentiment']=='Positivo']
tweets_negative = df[df['sentiment']=='Negativo']

In [93]:
tweets = df
tweets.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


# Funções de pré-processamento dos dados

In [92]:
##Funções que podem ajudar a criar um processo de análise de sentimento com outros modelos.

In [ ]:
def removestopwords(texto):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

In [ ]:
def stemming(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for i in texto.split():
        palavras.append(stemmer.stem(i))
    return (" ".join(palavras))

In [ ]:
def clean_data(texto):
    texto = re.sub(r'http\S+', '', texto).lower().replace('.', '').replace(';', '').replace(',', '')
    return texto

# Utilizando a ajuda do tokenizador de tweets do NLTK

In [95]:
from nltk.tokenize import TweetTokenizer

In [96]:
tweet_tokenizer = TweetTokenizer()

# Criando scores de polarização com o modelo Vader

In [98]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [61]:
analyzer = SentimentIntensityAnalyzer()

In [99]:
scores = []
for i in tweets['tweet_text']:
    sentiment = analyzer.polarity_scores(i)
    scores.append(sentiment['compound'])
tweets['scores'] = scores

In [100]:
tweets.head()

,id,tweet_text,tweet_date,sentiment,query_used,scores
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:),0.4588
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:),0.6705
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:),0.4588
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:),0.6514
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:),0.7284


In [101]:
tweets['Sentimento'] = tweets['scores'].apply(lambda x: 'Positivo' if x>=0 else 'Negativo')

In [102]:
tweets.head()

,id,tweet_text,tweet_date,sentiment,query_used,scores,Sentimento
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:),0.4588,Positivo
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:),0.6705,Positivo
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:),0.4588,Positivo
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:),0.6514,Positivo
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:),0.7284,Positivo


In [107]:
cond = df['sentiment']==df['Sentimento']
tweets['Equal'] = cond

,id,tweet_text,tweet_date,sentiment,query_used,scores,Sentimento,Equal
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:),0.4588,Positivo,True
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:),0.6705,Positivo,True
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:),0.4588,Positivo,True
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:),0.6514,Positivo,True
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:),0.7284,Positivo,True
...,...,...,...,...,...,...,...,...
785809,1050705141207367680,Acordar 8 horas é tão bom :),Fri Oct 12 11:10:01 +0000 2018,Positivo,:),0.4588,Positivo,True
785810,1050706655049109504,"@mayckcunha Olá, Mayck. Você já é cliente Clar...",Fri Oct 12 11:16:02 +0000 2018,Positivo,:),0.2869,Positivo,True
785811,1050705846907392005,Opa tava na merda mm e fiquei logo mais feliz ...,Fri Oct 12 11:12:49 +0000 2018,Positivo,:),0.4588,Positivo,True
785812,1050705490232127489,@andrebraga2806 Foi como a tua lealdade :),Fri Oct 12 11:11:24 +0000 2018,Positivo,:),0.4588,Positivo,True


In [ ]:
pct = (tweets['Equal'].mean())*100
pct

# Resultado final dos testes

In [110]:
print("A precisão da análise de sentimento utilizada é de: ", pct, "%")

A precisão da análise de sentimento utilizada é de:  85.54958912292582 %
